In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os 
import sys
from datetime import datetime, timedelta
from modeling.utils import process_address
from modeling.utils import * 
import json
import warnings
from multiprocessing import Pool
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

## Select the time window to extract house information

In [2]:
daily_scrpe_folder = '/home/user/DataCenter/HAR_data/combined_pickles/'
start_date = datetime(2024, 11, 7)
end_date = datetime(2024, 12,18)
daily_df = None 

for day in tqdm(range((end_date - start_date).days + 1)):
    cur_date = start_date + timedelta(days=day)
    date_str = cur_date.strftime("%Y_%m_%d")
    daily_pickle = os.path.join(daily_scrpe_folder, "zipcode_search__"+date_str+"_for_sale.pkl")
    try:
        if daily_df is None:
            daily_df = pd.read_pickle(daily_pickle)
            daily_df = daily_df[daily_df["status"].apply(lambda x: "contract" in x.lower())]
            daily_df["address_key"] = daily_df["address"].apply(lambda x: process_address(x))
            daily_df["date"] = date_str
        else:
            tem_df = pd.read_pickle(daily_pickle)
            tem_df = tem_df[tem_df["status"].apply(lambda x: "contract" in x.lower())]
            tem_df["address_key"] = tem_df["address"].apply(lambda x: process_address(x))
            tem_df["date"] = date_str
            daily_df = pd.concat([daily_df, tem_df], axis=0)\
                        .drop_duplicates("address", keep="last")
    except:
        pass 


00%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:23<00:00,  1.82it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 368/368 [04:02<00:00,  1.52it/s]


In [3]:
## Remove training set addresses
raw_df = pd.read_pickle("../../data/property_structured_12162024.pkl").reset_index(drop=True)
raw_df.head(2)

,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path,json_size,image_cnt,LAT,LON,building_sqft2,...,Baths,Maintenance Fee,Tax Rate,tax_rate2,Recent Market Value,Recent Tax Value,Property Type,County,Private Pool,Area Pool,Private_pool_desc,Pool_feature_multiclass,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,floor_type_multiclass,Exterior Type_multiclass,Exterior_multiclass,Style_multiclass,finance_option_multiclass,high_school_star,middle_school_star,elementary_school_star,school_org,general_desc
0,"4513 Refugio Dr, Plano, TX 75024",https://www.har.com//homedetail/4513-refugio-d...,"4 bedrooms 3,395 Sqft. ($233/Sqft.) 3 full & 1...",Karla Davis Fathom Realty,789990.0,66.0,Under Contract - P,75024,4.0,0.0,0.0,3395.0,0.0,0.0,0.0,"4513 Refugio Dr, Plano, TX 75024",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1690411643,/home/user/DataCenter/HAR_data/house_images/00...,4425,12,-96.782005,33.102204,3395.0,...,3.5,65.083333,1.864,NaN,157533.0,157533.0,single family,Collin County,No,None,None,[],[Slab],[Attached],[composition],"[carpet, ceramic tile, wood]","[brick, rock/stone]",[covered patio/porch],[traditional],[],NaN,5.0,5.0,[FRISCO ISD],"MOVE IN READY! Welcome to The Bartlet, this ge..."
1,"1516 Bay Area Blvd P12, Houston, TX 77058",https://www.har.com//homedetail/1516-bay-area-...,1 bedrooms 684 Sqft. ($117/Sqft.) 1 full baths...,Laurie Long RE/MAX Space Center,79900.0,79.0,Under Contract - PS,77058,1.0,0.0,0.0,684.0,0.0,0.0,0.0,"1516 Bay Area Blvd P12, Houston, TX 77058",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1689375459,/home/user/DataCenter/HAR_data/house_images/00...,5661,11,-95.114098,29.562197,684.0,...,1.0,272.000000,2.444,2.4437,69024.0,47052.0,townhouse/condo,Harris County,No,Yes,None,[],[Slab],[],[composition],"[tile, vinyl]","[brick, stucco]","[balcony, partially fenced, storage]",[traditional],"[cash sale, conventional, fha, investor, va]",5.0,NaN,4.0,[CLEAR CREEK ISD],This is the perfect starter home! EVERYTHING U...


In [4]:
train_address_key_set = set(raw_df["address_key"].values)
print(len(daily_df))
daily_df = daily_df[daily_df["address_key"].apply(lambda x: x not in train_address_key_set)]
print(len(daily_df))

43762
20430


In [5]:
print(len(daily_df))
daily_df.head(2)

20430


,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date
223,"13200 Bigelow Ln, Frisco, TX 75035",https://www.har.com//homedetail/13200-bigelow-...,"5 bedrooms 4,786 Sqft. ($271/Sqft.) 5 full & 1...",Russell Thornton Powerstar Realty,1299000.0,34.0,Under Contract - OP,75035,5.0,0.0,0.0,4786.0,0.0,0.0,0.0,"13200 Bigelow Ln, Frisco, TX 75035",2024_11_07
28,"1914 Forest Hill Blvd, Houston, TX 77023",https://www.har.com//homedetail/1914-forest-hi...,"3 bedrooms 1,444 Sqft. ($166/Sqft.) 1 full & 1...",Raul Diaz MetroPlus Realty,240000.0,20.0,Under Contract - OP,77023,3.0,0.0,0.0,1444.0,0.0,0.0,0.0,"1914 Forest Hill Blvd, Houston, TX 77023",2024_11_08


## Append full house json file 

In [6]:
json_folder = "/home/user/DataCenter/HAR_data/address_fullinfo_for_sale/"
all_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.endswith(".json")]
json_path_df = pd.DataFrame(all_files, columns=["json_path"])
json_path_df["address"] = json_path_df["json_path"].apply(lambda x: x.split("/")[-1][:-16])
json_path_df["time_epoch"] = json_path_df["json_path"].apply(lambda x: x.split("/")[-1][-15:-5])
print(len(json_path_df))
json_path_df = json_path_df.sort_values("time_epoch").drop_duplicates("address", keep="last")
print(len(json_path_df))
json_path_df.sample(3)

1709087
1335016


,json_path,address,time_epoch
1650684,/home/user/DataCenter/HAR_data/address_fullinf...,668 Cornelius Rd R...,1730737212
965014,/home/user/DataCenter/HAR_data/address_fullinf...,"4629 Coles Manor Pl, Dallas, TX 75204",1714197656
236445,/home/user/DataCenter/HAR_data/address_fullinf...,"7886 Minglewood Ln, Dallas, TX 75231",1611958516


In [7]:
join_df = daily_df.join(json_path_df.set_index("address"), on="address", how="inner")
print(len(join_df))
join_df.head(2)

20430


,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch
223,"13200 Bigelow Ln, Frisco, TX 75035",https://www.har.com//homedetail/13200-bigelow-...,"5 bedrooms 4,786 Sqft. ($271/Sqft.) 5 full & 1...",Russell Thornton Powerstar Realty,1299000.0,34.0,Under Contract - OP,75035,5.0,0.0,0.0,4786.0,0.0,0.0,0.0,"13200 Bigelow Ln, Frisco, TX 75035",2024_11_07,/home/user/DataCenter/HAR_data/address_fullinf...,1729229387
28,"1914 Forest Hill Blvd, Houston, TX 77023",https://www.har.com//homedetail/1914-forest-hi...,"3 bedrooms 1,444 Sqft. ($166/Sqft.) 1 full & 1...",Raul Diaz MetroPlus Realty,240000.0,20.0,Under Contract - OP,77023,3.0,0.0,0.0,1444.0,0.0,0.0,0.0,"1914 Forest Hill Blvd, Houston, TX 77023",2024_11_08,/home/user/DataCenter/HAR_data/address_fullinf...,1729050124


## Append Image Folder 

In [8]:
image_folder = "/home/user/DataCenter/HAR_data/house_images/"
all_folders = []
for i in tqdm(range(100)):
    for j in range(100):
        folder = os.path.join(image_folder, f"{i:04d}/{j:04d}") 
        all_folders.extend([os.path.join(folder, f) for f in os.listdir(folder)])


00%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.08it/s]

In [9]:
image_df = pd.DataFrame(all_folders, columns=["image_path"])
image_df["address"] = image_df["image_path"].apply(lambda x: x.split("/")[-1][:-11])
image_df["time_epoch"] = image_df["image_path"].apply(lambda x: x.split("/")[-1][-10:])
print(len(image_df))
image_df = image_df.sort_values("time_epoch").drop_duplicates("address", keep="last")
print(len(image_df))
image_df.sample(2)

608920
606158


,image_path,address,time_epoch
557526,/home/user/DataCenter/HAR_data/house_images/00...,"11503 Barrington Way, Austin, TX 78759",1733677704
538744,/home/user/DataCenter/HAR_data/house_images/00...,"7419 CEDAR FARM, San Antonio, TX 78239",1731128104


In [10]:
print(len(join_df))
join_df2 = join_df.join(image_df.set_index("address")[["image_path"]], on="address", how="inner")
print(len(join_df2))
join_df2.head(2)

20430
19547


,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path
223,"13200 Bigelow Ln, Frisco, TX 75035",https://www.har.com//homedetail/13200-bigelow-...,"5 bedrooms 4,786 Sqft. ($271/Sqft.) 5 full & 1...",Russell Thornton Powerstar Realty,1299000.0,34.0,Under Contract - OP,75035,5.0,0.0,0.0,4786.0,0.0,0.0,0.0,"13200 Bigelow Ln, Frisco, TX 75035",2024_11_07,/home/user/DataCenter/HAR_data/address_fullinf...,1729229387,/home/user/DataCenter/HAR_data/house_images/00...
28,"1914 Forest Hill Blvd, Houston, TX 77023",https://www.har.com//homedetail/1914-forest-hi...,"3 bedrooms 1,444 Sqft. ($166/Sqft.) 1 full & 1...",Raul Diaz MetroPlus Realty,240000.0,20.0,Under Contract - OP,77023,3.0,0.0,0.0,1444.0,0.0,0.0,0.0,"1914 Forest Hill Blvd, Houston, TX 77023",2024_11_08,/home/user/DataCenter/HAR_data/address_fullinf...,1729050124,/home/user/DataCenter/HAR_data/house_images/00...


In [11]:
join_df2[["address", "json_path", "image_path"]].sample(1).iloc[0].values

array(['1030 Lehman St, Houston, TX 77018',
       '/home/user/DataCenter/HAR_data/address_fullinfo_for_sale/1030 Lehman St, Houston, TX 77018_1732277128.json',
       '/home/user/DataCenter/HAR_data/house_images/0091/0034/1030 Lehman St, Houston, TX 77018_1732252624'],
      dtype=object)

In [12]:
## Strategy to drop the redudant addresses, chooose the one with largest json size, and the most images in the folder
join_df2["json_size"] = join_df2["json_path"].apply(lambda x: os.path.getsize(x))

In [13]:
join_df2["image_cnt"] = join_df2["image_path"].apply(lambda x: len([_ for _ in os.listdir(x) if _.endswith(".jpeg")]))

In [14]:
## Defer the drop duplicates to the end
final_df = join_df2.sort_values(["json_size", "image_cnt"]).drop_duplicates("address_key", keep="last")
print(len(final_df), len(join_df2))

19547 19547


## Get feature processing from json file

In [22]:
def read_json(json_path):
    with open(json_path) as f:
        return json.load(f)

processed_df = []
n_rows = len(final_df)
batch_size = 5000
n_batches = (n_rows + batch_size - 1)// batch_size

for batch_i in tqdm(range(n_batches)):
    sample_df = final_df.iloc[batch_i * batch_size: (batch_i + 1) * batch_size]
    sample_df["content"] = sample_df["json_path"].apply(read_json)
    sample_df["LAT"], sample_df["LON"] = zip(*sample_df["content"].apply(lambda x: get_lon_lat(x["coords"][0], x["coords"][1])))
    sample_df["building_sqft2"] = sample_df["content"].apply(lambda x: parse_built_Sqft(x))
    sample_df["building_sqft"] = sample_df.apply(lambda x: x["building_sqft2"] if np.isnan(x["building_sqft2"]) is False else x["building_sqft"], axis=1)
    sample_df["Lot Size"] = sample_df["content"].apply(lambda x: parse_lot_size(x)[0])
    sample_df["Year Built"] = sample_df["content"].apply(lambda x: parse_built_year(x)[0])
    sample_df["Garage Number"] = sample_df["content"].apply(lambda x: get_garage_num(x))
    sample_df["Bedrooms"] = sample_df["content"].apply(lambda x: parse_bedroom(x))
    sample_df["Baths"] = sample_df["content"].apply(lambda x: parse_bath(x))
    sample_df["Maintenance Fee"] = sample_df["content"].apply(lambda x: parse_maintenance_fee(x))
    sample_df["Tax Rate"] = sample_df["content"].apply(lambda x: extract_tax_rate_tax_table(x["house_tax_table"]))
    sample_df["tax_rate2"] = sample_df["content"].apply(get_tax_rate)
    sample_df.loc[sample_df["Tax Rate"].isnull(), "Tax Rate"] = sample_df.loc[sample_df["Tax Rate"].isnull(), "tax_rate2"]
    sample_df["Recent Market Value"] = sample_df["content"].apply(lambda x: get_recent_market_value(x))
    sample_df["Recent Tax Value"] = sample_df["content"].apply(lambda x: get_recent_tax_value(x))
    sample_df["Property Type"] = sample_df["content"].apply(lambda x: parse_property_type(x))
    sample_df["County"] = sample_df["content"].apply(lambda x: x["house_features"]["County:"] if "County:" in x["house_features"] else None)
    sample_df["Private Pool"] = sample_df["content"].apply(lambda x: private_pool_feature(x))
    sample_df["Area Pool"] = sample_df["content"].apply(lambda x: area_pool_feature(x))
    sample_df["Private_pool_desc"] = sample_df["content"].apply(lambda x: private_pool_desc_feature(x))
    sample_df["Pool_feature_multiclass"] = sample_df["content"].apply(lambda x: private_pool_multiclass_feature(x))
    sample_df["Foundation_multiclass"] = sample_df["content"].apply(lambda x: Foundation_multiclass_feature(x))
    sample_df["Garage Types_multiclass"] = sample_df["content"].apply(lambda x: Garage_Types_multiclass_features(x))
    sample_df["Roof Type_multiclass"] = sample_df["content"].apply(lambda x: Roof_Type_multiclass(x))
    sample_df["floor_type_multiclass"] = sample_df["content"].apply(lambda x: floor_type_multiclass(x))
    sample_df["Exterior Type_multiclass"] = sample_df["content"].apply(lambda x: exterior_type_multiclass(x))
    sample_df["Exterior_multiclass"] = sample_df["content"].apply(lambda x: exterior_multiclass(x))
    sample_df["Style_multiclass"] = sample_df["content"].apply(lambda x: style_multiclass(x))
    sample_df["finance_option_multiclass"] = sample_df["content"].apply(lambda x: finanace_option_multiclass(x))
    sample_df["high_school_star"] = sample_df["content"].apply(lambda x: extract_high_school_name(x["house_schools"])[2] if "house_schools" in x else np.nan)
    sample_df["middle_school_star"] = sample_df["content"].apply(lambda x: extract_mid_school_name(x["house_schools"])[2] if "house_schools" in x else np.nan)
    sample_df["elementary_school_star"] = sample_df["content"].apply(lambda x: extract_elemetary_school_name(x["house_schools"])[2] if "house_schools" in x else np.nan)
    sample_df["school_org"] = sample_df["content"].apply(lambda x: school_org(x))
    sample_df["general_desc"] = sample_df["content"].apply(lambda x: x["general_desc"] if "general_desc" in x else None)
    processed_df.append(sample_df)


00%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.16s/it]

In [23]:
processed_final_df = pd.concat(processed_df)

In [25]:
processed_final_df_filtered = processed_final_df.drop_duplicates(subset=["address_key"], keep="first").drop(columns=["content"])

In [28]:
processed_final_df_filtered.sample(3)

,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path,json_size,image_cnt,LAT,LON,building_sqft2,...,Baths,Maintenance Fee,Tax Rate,tax_rate2,Recent Market Value,Recent Tax Value,Property Type,County,Private Pool,Area Pool,Private_pool_desc,Pool_feature_multiclass,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,floor_type_multiclass,Exterior Type_multiclass,Exterior_multiclass,Style_multiclass,finance_option_multiclass,high_school_star,middle_school_star,elementary_school_star,school_org,general_desc
42412,"1615 King Ranch Rd, Conroe, TX 77301",https://www.har.com//homedetail/1615-king-ranc...,"4 bedrooms 2,316 Sqft. ($151/Sqft.) 3 full bat...",Patrick McGrath Meritage Homes Realty,350120.0,1.0,Under Contract - P,77301,4.0,0.0,0.0,2316.0,0.0,0.0,0.0,"1615 King Ranch Rd, Conroe, TX 77301",2024_12_18,/home/user/DataCenter/HAR_data/address_fullinf...,1731510890,/home/user/DataCenter/HAR_data/house_images/00...,4211,3,-95.436558,30.275588,2316.0,...,3.0,41.666667,2.65,2.6500,NaN,NaN,single family,Montgomery County,No,No,None,[],[Slab],[Attached],[composition],"[tile, vinyl]","[brick, cement board, stone, wood]","[back yard fenced, patio/deck]",[ranch],"[cash sale, conventional, fha, va]",3.0,NaN,4.0,[CONROE ISD],"Brand new, energy-efficient home available by ..."
74375,"7110 Echo Pines Dr, Humble, TX 77346",https://www.har.com//homedetail/7110-echo-pine...,"3 bedrooms 1,640 Sqft. ($137/Sqft.) 2 full bat...",Cheryl Stevens JLA Realty,225000.0,19.0,Under Contract - P,77346,3.0,0.0,0.0,1640.0,0.0,0.0,0.0,"7110 Echo Pines Dr, Humble, TX 77346",2024_12_18,/home/user/DataCenter/HAR_data/address_fullinf...,1730000180,/home/user/DataCenter/HAR_data/house_images/00...,5002,26,-95.169682,29.989437,1640.0,...,2.0,38.750000,2.08,2.0802,240438.0,218755.0,single family,Harris County,No,Yes,None,[],[Slab],[Attached],[composition],"[carpet, laminate, tile]","[brick, cement board, wood]","[back yard fenced, patio/deck]",[traditional],"[cash sale, conventional, fha]",4.0,3.0,0.0,[HUMBLE ISD],Come check out this great split floorpan with ...
6438,"1422 Bertha Branch, San Antonio, TX 78224",https://www.har.com//homedetail/1422-bertha-br...,"4 bedrooms 1,600 Sqft. ($144/Sqft.) 2 full bat...",Christopher Marti Housifi,229999.0,40.0,Under Contract - P,78224,4.0,0.0,0.0,1600.0,0.0,0.0,0.0,"1422 Bertha Branch, San Antonio, TX 78224",2024_12_18,/home/user/DataCenter/HAR_data/address_fullinf...,1730086555,/home/user/DataCenter/HAR_data/house_images/00...,2609,1,-98.528918,29.311755,1600.0,...,2.0,73.333333,NaN,NaN,NaN,NaN,single family,Bexar County,No,None,None,[],[Slab],[],[composition],"[carpet, vinyl]",[cement board],[],[],"[cash sale, conventional, fha, tvlb, va]",NaN,NaN,NaN,[],The Pinehollow- This single-level home showcas...


## Run Text encoding and Image encoding

In [29]:
## Get the list of images to be encoded
def get_image_paths(image_folder, check_mean=False):
    images_path = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith(".jpeg")]
    n_images = len(images_path)
    if check_mean:
        images_path = [f_ for f_ in images_path if os.path.exists(f_.replace(".jpeg", "mean.pt")) is False] 
    else:
        images_path = [f_ for f_ in images_path if os.path.exists(f_.replace(".jpeg", ".pt")) is False]
    return images_path, n_images 

batch_size = 2048 
n_batches = len(processed_final_df_filtered) // batch_size + 1
all_img_cnt = 0
all_img_paths = []

for batch_i in tqdm(range(n_batches)):
    with Pool(10) as p:
        img_path, img_cnt = zip(*p.map(get_image_paths, processed_final_df_filtered["image_path"].iloc[batch_i * batch_size: (batch_i + 1) * batch_size].values))
    [all_img_paths.extend(path_) for path_ in img_path]
    all_img_cnt += sum(img_cnt)
print(len(all_img_paths))


00%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]

In [31]:
## Run DINO Encoding
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

device = "cuda:0"
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')
model.cuda(device)

def read_image(image_path):
    image = Image.open(image_path)
    return image

In [32]:
batch_size = 1024 
n_batches = (len(all_img_paths) + batch_size - 1) // batch_size

for batch_i in tqdm(range(n_batches)):
    batch_images = all_img_paths[batch_i * batch_size: batch_i * batch_size + batch_size]
    with Pool(10) as p:
        inp_imgs = p.map(read_image, batch_images)
    inputs = processor(images=inp_imgs, return_tensors="pt").to(device, non_blocking=True)
    with torch.no_grad():
        last_hidden_states = model(**inputs)[0]
        embedding_list = last_hidden_states[:, 0, :].cpu()
        mean_embedding_list = torch.mean(last_hidden_states, axis=1).cpu()
    embedding_list = [e.clone() for e in embedding_list]
    # mean_embedding_list = [e.clone() for e in mean_embedding_list]
    _ = [torch.save(embedding_list[i], batch_images[i].replace(".jpeg", ".pt")) for i in range(len(embedding_list))]
    # _ = [torch.save(mean_embedding_list[i], batch_images[i].replace(".jpeg", "mean.pt")) for i in range(len(embedding_list))]


00%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 252/252 [42:52<00:00, 10.21s/it]

In [33]:
def load_and_stack_tensor(image_folder, use_mean_tensor=False):
    if use_mean_tensor:
        pt_paths = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith("mean.pt")]
    else: 
        pt_paths = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith(".pt") and "mean" not in f_]
    if len(pt_paths) == 0:
        return np.zeros(384)
    pt_list = np.array([torch.load(pt_path).numpy() for pt_path in pt_paths])
    return np.mean(pt_list, axis=0)

with Pool(10) as p:
    image_dinov2s_cls_mean = p.map(load_and_stack_tensor, processed_final_df_filtered["image_path"].values)

In [34]:
processed_final_df_filtered["img_emb"] = image_dinov2s_cls_mean

#### Text Embedding

In [35]:
from transformers import pipeline
feature_extraction = pipeline('feature-extraction', model="roberta-base", tokenizer="roberta-base", device=0, truncation=True)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [37]:
cnt = 0 
general_desc_mean_features = []
for desc in tqdm(processed_final_df_filtered["general_desc"].values):
    if desc is None:
        cnt += 1
        general_desc_mean_features.append(np.zeros(768))
    else: 
        feature = feature_extraction(desc,  return_tensors="pt")
        general_desc_mean_features.append(np.mean(feature[0].clone().numpy(), axis=0))
processed_final_df_filtered["general_desc_roberta_mean_features"] = general_desc_mean_features

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset

00%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19547/19547 [05:29<00:00, 59.25it/s]

In [38]:
processed_final_df_filtered.to_pickle("../../data/property_structured_test_1107_1216_2024.pkl")

In [43]:
path = "../../data/property_structured_12162024_tabformer_t32c.pt"
def read_column_name_and_stats(path):
    with open(path, "rb") as f:
        _, col_stats = torch.load(f)
    col_names = list(col_stats.keys())
    return col_names, col_stats
col_names, col_stats = read_column_name_and_stats(path)

In [42]:
processed_final_df_filtered[col_names].sample(3)

,LON,LAT,building_sqft,Lot Size,Year Built,Garage Number,Bedrooms,Baths,Maintenance Fee,Tax Rate,Recent Market Value,Recent Tax Value,elementary_school_star,middle_school_star,high_school_star,date,price,status,Property Type,County,Private Pool,Area Pool,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,Pool_feature_multiclass,floor_type_multiclass,finance_option_multiclass,Exterior Type_multiclass,Style_multiclass,school_org,img_emb,general_desc_roberta_mean_features
100,33.064635,-96.852624,6124.0,987.0,2004.0,3.0,5.0,5.0,75.0,1.737,1904221.0,1136687.0,NaN,NaN,5.0,2024_11_23,1749900.0,Under Contract - OP,single family,Denton County,Yes,None,[Slab],[Attached],[spanish tile],"[gunite, in ground, outdoor pool, water feature]","[carpet, ceramic tile, hardwood]",[],[stucco],"[mediterranean, traditional]",[LEWISVILLE ISD],"[0.683101, -0.27007037, -0.6161824, 3.191913, ...","[-0.05843798, 0.25097364, 0.024428194, -0.1755..."
14989,32.319061,-95.472874,1653.0,688.0,2024.0,2.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2024_12_18,275990.0,Under Contract - P,single family,Henderson County,No,None,[Slab],[Attached],[composition],[],"[carpet, laminate]",[],[brick],[traditional],[BROWNSBORO ISD],"[-0.46461964, 0.46970117, -3.2459972, -0.17523...","[-0.041528974, 0.105753265, 0.028724108, -0.13..."
49889,33.054813,-96.308534,1499.0,3480.0,1994.0,2.0,3.0,2.0,NaN,1.956,310974.0,271140.0,0.0,NaN,4.0,2024_12_14,265000.0,Under Contract - PS,single family,Collin County,No,None,[Slab],[Attached],[metal],[],"[carpet, ceramic tile]",[],"[brick, siding]",[traditional],[COMMUNITY ISD],"[0.31914845, -0.2120942, -0.47378743, 1.812268...","[-0.016346423, 0.213551, 0.035406973, -0.11352..."


In [46]:
col_names

['LON',
 'LAT',
 'building_sqft',
 'Lot Size',
 'Year Built',
 'Garage Number',
 'Bedrooms',
 'Baths',
 'Maintenance Fee',
 'Tax Rate',
 'Recent Market Value',
 'Recent Tax Value',
 'elementary_school_star',
 'middle_school_star',
 'high_school_star',
 'date',
 'price',
 'status',
 'Property Type',
 'County',
 'Private Pool',
 'Area Pool',
 'Foundation_multiclass',
 'Garage Types_multiclass',
 'Roof Type_multiclass',
 'Pool_feature_multiclass',
 'floor_type_multiclass',
 'finance_option_multiclass',
 'Exterior Type_multiclass',
 'Style_multiclass',
 'school_org',
 'img_emb',
 'general_desc_roberta_mean_features']